In [ ]:
# To use only Google Colab
# ! pip install matplotlib --upgrade

## Modelos de regresión con variables categóricas

Los modelos lineales que vimos antes
  - Solo pueden trabajar con variables dependientes e independientes continuas.

Existen modelos alternartivos para trabajar con variables categóricas
  - En las variables independientes
  - En las variables dependientes

## Análisis de Regresión Logística con variables independientes categóricas

Es posible hacer un modelo de regresión lineal con variables independientes
categóricas.

Tenemos que modificar los datos y el modelo, para que las variables categóricas
pueden ser incluidas.

- Si la variable es dicotómica:
  - Se modifica su valor para sea
    - 1 para una categroría
    - 0 para la otra categoria.
- Si la variable tiene tres o mas valores:
  - Se incluyen en el modelo
    - Tantas variables como categorias
    - Cada variable puede tener valores 0 o 1


In [ ]:
from rdatasets import data, descr

survey = data("MASS", "survey")
survey.dropna(inplace=True)
print(survey)
print(descr("MASS", "survey"))

Vamos a tratar de predecir la altura:
  - Por el sexo la persona
  - La frecuencia de fumar
  - La frecuencia de ejercitación


In [ ]:
import pandas as pd
survey_mod = survey[["Height", "Sex", "Smoke", "Exer"]]

survey_mod = pd.get_dummies(survey_mod)
survey_mod.drop(columns=["Sex_Male"], inplace = True)
survey_mod

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np


train_frac = 0.7
train_elems = int(round(train_frac * len(survey_mod)))
print(f"Selecciono {train_elems} elementos para entrenar")
print(f"Selecciono {len(survey_mod)-train_elems} elementos para testear")

suffled_indexes = survey_mod.index.to_numpy()
np.random.shuffle(suffled_indexes)
train_indexes = suffled_indexes[:train_elems]
test_indexes = suffled_indexes[train_elems:]

survey_train = survey_mod.loc[train_indexes,:]
survey_test = survey_mod.loc[test_indexes,:]

exog = survey_train.drop(columns=["Height"])
exog = sm.add_constant(exog)
exog

regmod = sm.OLS(
  endog = survey_train["Height"],
  exog = exog
)
fitted = regmod.fit()
fitted.params

test = survey_test.drop(columns=["Height"])
test = sm.add_constant(test)
test

test_predicted = fitted.predict(test)
test_predicted.name = "Predicted"
test_predicted

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

plt.scatter(
  x = survey_test["Height"],
  y = test_predicted
)
plt.ylabel("Altura predicha")
plt.xlabel("Altura real")

pearson_corr = st.pearsonr(survey_test["Height"], test_predicted)
pearson_corr


fitted2 = sm.OLS(
  survey_test["Height"],
  sm.add_constant(test_predicted)
).fit()
x = np.linspace(min(survey_test["Height"]), max(survey_test["Height"]), 100)
plt.plot(
  x,
  fitted2.params @ (sm.add_constant(x)).T,
  color = "Red"
)
plt.text(155, 190, f"$\\rho = {pearson_corr[0]:0.3f}$")


## Modelo lineal con una variable dependiente con varias categorías

Para utilizar un modelo lineal para prededir

- Una variable dependiente
  - Can varias categorías
- Se contruyen varios modelos logisticos
  - Si hay N categorias,
  - Se generan N modelos.
  - Cada uno de ellos predice cada variable.
  - Finalmente,
    - Se toma el valor de la mayor predicción individual

Queremos predecir la frecuencia de ejericio:
- El pulso
- La altura
- Edad

In [ ]:
outputs = survey["Exer"].unique()
outputs

In [ ]:
from rdatasets import data, descr

survey = data("MASS", "survey")
survey.dropna(inplace=True)

train_frac = 0.7
train_elems = int(round(train_frac * len(survey)))
print(f"Selecciono {train_elems} elementos para entrenar")
print(f"Selecciono {len(survey)-train_elems} elementos para testear")

suffled_indexes = survey_mod.index.to_numpy()
np.random.shuffle(suffled_indexes)
train_indexes = suffled_indexes[:train_elems]
test_indexes = suffled_indexes[train_elems:]

survey_train = survey.loc[train_indexes,:]
survey_test = survey.loc[test_indexes,:]
survey_test
exog = survey_train[["Pulse", "Height", "Age"]]
exog

fitted_models = []
for o in outputs:
  c_endog = (survey_train["Exer"] == o).astype(int)
  m = sm.Logit(
    endog = c_endog,
    exog = exog
  )
  fitted_models.append(m.fit())

In [ ]:
from sklearn.metrics import classification_report

exog = survey_test[["Pulse", "Height", "Age"]]

predicted = [
  m.predict(exog)
  for m in fitted_models
]

predicted = np.column_stack(predicted)
joint_predicted = np.apply_along_axis(np.argmax, 1, predicted)

test_values = survey_test["Exer"].replace(
  {
    outputs[0]: 0,
    outputs[1]: 1,
    outputs[2]: 2
  }
)

a = classification_report(
  test_values,
  joint_predicted,
  target_names = outputs,
  zero_division = 0
)
print(a)